<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/korean_abstractive_summarizaion_v1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Korean Summarizer Using Multiple Discriminators**

참조 : https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch

* 2020년12월27일 v1.0 완전히 실패...
* Generator 새로 제작!

In [227]:
DO_ALL = True # 전체 실행하면서 시간 걸리는 걸 Pass 하려면 이걸 False ...


In [177]:

if DO_ALL:
    !pip install sentence-transformers==0.3.0
    !pip install transformers==3.0.2
    !pip install wikipedia
    !pip install konlpy

In [178]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.framework import tensor_shape
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 학습을 위한 데이터 준비

네이버 뉴스에서 아무거나 하나 Text를 얻어옴

이것을 '요약' 목표

In [179]:
org_text = """주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.
주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 '군사작전'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 '패스트트랙'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.
이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 '공수처는 야당의 동의 없이는 절대 출범할 수 없는 것'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 '비토권'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.
주 원내대표는 "이해찬 전 민주당 대표가 얘기한 '민주당 20년 집권'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 '민주주의 없는 민주당'이 법위에 군림하는 '반민주'를 거리낌없이 획책하는 것"이라고 언급했다.
그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.
주 원내대표는 "문 대통령은 제게 '공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다'고 했는데, 그런 분이 청와대와 대통령 주변을 감시하는 특별감찰관은 취임 이후 지금까지 왜 임명하지 않았는가"라며 "공수처는 권력형 비리의 쓰레기 하치장, 종말 처리장이 될 것"이라고 비판했다.
문재인 정부를 향해 주 원내대표는 "문 대통령과 그 사도들은 법치가 미치지 않는 무오류의 화신이 될 것"이라며 "오류를 인정하지 않는 존재가 바로 신이며 그 아래에는 자신들의 지도자를 목숨바쳐 지킴으로서 정의를 실현하겠다는 추종자들로 넘쳐 난다. 공수처는 지도자의 신성을 인정하지 않는 세력을 정죄하는 수단으로 전락할 것"이라고 질타했다.
주 원내대표는 "저도 법조인이지만 대통령과 공수처장이 마음대로 검사들과 수사관들을 임명하는 이 끔찍한 사법기구가 어떤 일을 할지 두렵기만 하다"며 "공수처는 검찰과 경찰 위에 있는 사법기구로, 헌법과 법으로 독립성을 보장하는 검찰총장을 이렇게 핍박하는 정권이 공수처를 어떻게 운영할지 불을 보듯 뻔한 일"이라고 예측했다.
그러면서 주 원내대표는 "추미애 법무장관을 앞장 세워 윤석열 검찰의 권력 비리 수사를 저지하려다가 난관에 봉착하자 무슨 수를 써서라도 공수처를 출범시키려 한다. 공수처장 자리에는 추미애보다 더 한 막무가내 내 편을 앉힐 게 분명한 것"이라며 "문 정권의 파렴치와 오만함을 최전선에서 온 몸으로 겪어온 저로서는 민주당이 내일부터 국회에서 보일 행태가 환히 보인다. 180석의 민주당이 또 군사작전을 개시하면 그걸 누가 막겠는가"라고 성토했다.
주 원내대표는 "공수처법을 막을 힘이 우리 야당에게는 없다. 삭발하고 장외투쟁해 봐야 눈 하나 깜짝할 사람들이 아닌 것"이라며 "대란대치(大亂大治), 세상을 온통 혼돈 속으로 밀어넣고 그걸 권력 유지에 이용한다는 게 이 정권의 통치기술"이라고 규탄했다.
아울러 주 원내대표는 "권력은 바람, 국민은 풀이다. 바람이 불면 청보리 밭의 보리가 눕는다"며 "권력은 풀들이 다시는 일어서지 못하도록 풀을 짓밟지만 풀들은 다시 일어난다. 시인 김수영은 '바람보다 먼저 눕지만, 바람보다 먼저 일어나는' 민초의 힘을 노래했다"고 말했다.
마지막으로 주 원내대표는 "문재인 정권은 이제 곧 국회에서 광장에서 짓밟힌 풀들이 일어서서 아우성치는 모습을 지켜보게 될 것"이라며 "대란대치를 끝장내려는 국민적 저항에 직면할 것"이라고 거듭 강조했다.
"""

In [180]:
org_text

'주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.\n주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 \'군사작전\'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 \'패스트트랙\'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.\n이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 \'공수처는 야당의 동의 없이는 절대 출범할 수 없는 것\'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 \'비토권\'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.\n주 원내대표는 "이해찬 전 민주당 대표가 얘기한 \'민주당 20년 집권\'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 \'민주주의 없는 민주당\'이 법위에 군림하는 \'반민주\'를 거리낌없이 획책하는 것"이라고 언급했다.\n그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.\n주 원내대표는 "문 대통령은 제게 \'공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다\'고 했는데, 그런 분이 청

한국어 문법체계에 따라 요약문을 생성하기 위해 한국어 문장 샘플을 준비

'한글 위키백과'에서 임의의 문장을 수집 함

In [181]:
#한국어 위키백과에서 스크랩핑

import wikipedia as wiki
wiki.set_lang('ko')

In [182]:
# '전래동화' 라는 keyword로 100개 page의 Text를 취득

def __search_from_wiki(question,max_rank):
    results = wiki.search(question,results=max_rank)
    print(results)
    contents = []
    for result in results:
        try:
            page = wiki.page(result)
            #print(f"Top wiki result: {page}")
            text = page.content
            ln = len(text)
            print(f'Collecting page : {page} , text length {str(ln)}')
            #if ln < 4000:
            #  contents.append(text)
            #else:
            #  contents.append(text[0:4000])
            contents.append(text)
        except Exception as ex:
          print(ex)
    return contents

if DO_ALL:
    ko_grammar_set_raw = __search_from_wiki("전래동화", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

전체 수집한 Page Count : 197


In [183]:

if DO_ALL:
    ko_grammar_set_raw += __search_from_wiki("역사", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

전체 수집한 Page Count : 197


In [184]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    return txt 

ko_grammar_set_raw = [clean_text(x) for x in ko_grammar_set_raw]
print('Sample text : ')
print('--------------------------------------------------------------------------------------------')
print(ko_grammar_set_raw[50])
print('--------------------------------------------------------------------------------------------')

Sample text : 
--------------------------------------------------------------------------------------------
김덕현(본명은 김덕구, 1967년 4월 19일 ~ )은 대한민국의 배우이다. 충청남도 보령에서 태어나 청소년기를 보냈다. 서울예술전문대학 방송연예학과를 졸업했다. 1991년 KBS 14기 공채 탤런트로 데뷔했다. 학력 서울예술전문대학 방송연예학과 졸업 출연작  드라마 KBS2 단막드라마 《TV 손자병법》(1992년)KBS2 단막드라마 《신 손자병법》(1993년)KBS2 수목드라마 《전설의 고향》(1996년 6월 26일 ~ 1996년 9월 12일) ... 구미호/망자의 소원 역KBS2 월화 미니시리즈 《거짓말》(1998년 3월 20일 ~ 1998년 6월 2일)KBS2 드라마 《부부클리닉 사랑과 전쟁》(1999년 10월 22일 ~ 2009년 4월 17일)KBS2 월화 미니시리즈 《학교》(1999년 2월 22일 ~ 1999년 4월 13일) ... 선생님 역MBC 단막극 《MBC 베스트극장》(1999년)iTV 일일드라마 《해바라기 가족》(2001년)MBC 법정드라마 《실화극장 죄와 벌》(2003년)KBS2 주말연속극 《애정의 조건》 (2004년 3월 20일 ~ 2004년 10월 10일)SBS 일일드라마 《소풍가는 여자》 (2004년 5월 3일 ~ 2004년 10월 8일)KBS1 일일연속극 《금쪽같은 내 새끼》(2004년 6월 7일 ~ 2005년 2월 11일)KBS1 대하드라마 《불멸의 이순신》(2004년 9월 4일 ~ 2005년 8월 28일) ... 언복 역KBS1 TV소설 《고향역》(2005년 8월 29일 ~ 2006년 3월 18일) ... 최 계장 역KBS1 대하드라마 《서울 1945》(2006년 1월 7일 ~ 2006년 9월 10일) - 박헌영 비서 역KBS2 아침드라마 《아줌마가 간다》(2006년 11월 13일 ~ 2007년 5월 19일) - 김성태 역KBS2 아침드라마

문장으로 잘라 낸다

In [185]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [186]:
#Split the document into sentences
ko_grammar_sentences = []
for document in ko_grammar_set_raw:
    ko_grammar_sentences += nltk.sent_tokenize(document)

print("Num sentences:", len(ko_grammar_sentences))

Num sentences: 13551


In [187]:
ko_grammar_sentences[300]

"현재 기탄교육은 학습지로는 '기탄국어, 한글떼기, 기탄한글, 기탄수학, 기탄사고력수학, 영단어 암기 끝!, 기탄 한자 빨리따기, 기탄중국어' 등이 있다."

In [188]:
f = open("ko_sentence_set.txt", "a")
for t in ko_grammar_sentences:
    f.write(t+'\n')
f.close()

형태소 분리하여 모든 문장을 형태소 Code로 변환 한다.

In [189]:
from konlpy.tag import Twitter
twitter = Twitter()

print(twitter.pos(ko_grammar_sentences[305]))

[('백운호수', 'Noun'), ('에서', 'Josa'), ('의왕시', 'Noun'), ('와', 'Josa'), ('의왕시', 'Noun'), ('축제', 'Noun'), ('추진', 'Noun'), ('위원회', 'Noun'), ('가', 'Josa'), ('공동', 'Noun'), ('으로', 'Josa'), ('주최', 'Noun'), ('하여', 'Verb'), ('매년', 'Noun'), ('9월', 'Number'), ('에', 'Foreign'), ('이틀', 'Noun'), ('동안', 'Noun'), ('열리는', 'Verb'), ('축제', 'Noun'), ('이다', 'Josa'), ('.', 'Punctuation')]


In [190]:
# 형태소 Code table의 구성

_MAX_MORP_LENGTH = 128
_PADDING_CODE = 0  # padding code
_MISMATCH_CODE = 1 # mismatch word code               ex) @@@
_MISMATCH_WORD = '@@@' # 이거 아래에서 쓴다.

morpheme_table = {}
morp_code = _MISMATCH_CODE+1
morpheme_table['Pad'] = _PADDING_CODE 
morpheme_table['Mst'] = _MISMATCH_CODE 
for sentence in ko_grammar_sentences[:1000]:
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        if morp in morpheme_table:
            pass
        else:
            morpheme_table[morp] = morp_code
            morp_code += 1

In [191]:
print('Korean morpheme code table')
print('----------------------------------------------------------')
print('  Morpheme        Code')
print('')
for morp in morpheme_table.keys():
    print(f' {morp.ljust(15)}   {morpheme_table[morp]}')
print('----------------------------------------------------------')


Korean morpheme code table
----------------------------------------------------------
  Morpheme        Code

 Pad               0
 Mst               1
 Noun              2
 Punctuation       3
 Foreign           4
 Josa              5
 Verb              6
 Modifier          7
 Adjective         8
 Suffix            9
 Adverb            10
 Number            11
 Alpha             12
 Conjunction       13
 Determiner        14
 VerbPrefix        15
 Exclamation       16
 KoreanParticle    17
 Eomi              18
 ScreenName        19
 URL               20
----------------------------------------------------------


In [192]:
# morpheme 코드 변환기
def morpheme_encode(sentence):
    encode=[]
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        encode.append(_MISMATCH_CODE if word==_MISMATCH_WORD else morpheme_table[morp])
    return encode

code = morpheme_encode('야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가')
print(f'Code length : {len(code)}')

Code length : 67


In [193]:
# 전체 형태소 코드로 변환
if DO_ALL:
    ko_grammar_set = []
    for sentence in ko_grammar_sentences:
        code = morpheme_encode(sentence)
        if len(code) <= _MAX_MORP_LENGTH:
            ko_grammar_set.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])

    ko_grammar_set = np.asarray(ko_grammar_set)
ko_grammar_set.shape

(13352, 128)

In [194]:
from numpy import savetxt
# save to csv file
savetxt('ko_morpheme_set.csv', ko_grammar_set, delimiter=',')


# Dataset 전체 구성

In [195]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer

if DO_ALL:
    # embedder download...
    embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

In [196]:
BATCH_SIZE = 64
BATCH_COUNT = 50

In [197]:
#dataset 다시 만듦

org_text_emb = embedder.encode([org_text])[0]
print(f'Text embedding shape : {org_text_emb.shape}')
dataset = []
for i in range(BATCH_COUNT):
    emb_batch_set = []
    cod_batch_set = []
    for j in range(BATCH_SIZE):
        emb_batch_set.append(org_text_emb)
        cod_batch_set.append(ko_grammar_set[BATCH_SIZE*i+j])

    emb_batch_set = np.asarray(emb_batch_set)
    cod_batch_set = np.asarray(cod_batch_set)
    dataset.append((emb_batch_set,cod_batch_set))

print(f'Total dataset count :{BATCH_COUNT*BATCH_SIZE}')

Text embedding shape : (1024,)
Total dataset count :3200


In [198]:
# 30번째 배치의 형태소코드셋 중 20번째꺼 확인
print(dataset[30][1][20])
print(dataset[31][1][20])

[2 9 2 5 2 2 5 6 2 2 5 6 2 2 5 2 5 6 3 2 5 2 5 2 6 2 5 6 2 5 6 6 2 8 3 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 2  2  5  2  5  2  5  2  2  2  5  2  8  3  2  5  2  6  6  2  5  2  5  3
 11  3 11  3 11  3 11  3 11  3 11  3 11  3  2  2  5  2  5  6  2  6  2  5
  6  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


# Generator 구현

In [199]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([org_text])
total_words = len(tokenizer.word_index)

print(f'Total token count of origin text : {total_words}')

Total token count of origin text : 394


원문을 40개의 token 으로 구성된 문장으로 요약 하는것....<br>
향후 LSTM으로 G 구성 필요

In [200]:

_MAX_GEN_TOKEN = 40
_NOISE_DIM = total_words

word_table = {}

for word,index in tokenizer.word_index.items():
    word_table[index-1] = word

current_token_len = len(word_table)



In [201]:
print('Token table of origin text')
print('---------------------------------------------')
print(' Code         Token      ')
print('')
for k in word_table.keys():
  print( f'  {str(k).ljust(8)}    {word_table[k]}')
print('---------------------------------------------')


Token table of origin text
---------------------------------------------
 Code         Token      

  0           원내대표는
  1           주
  2           것
  3           이라고
  4           며
  5           문재인
  6           않는
  7           문
  8           이라며
  9           이제
  10          풀들이
  11          수
  12          대통령과
  13          공수처는
  14          될
  15          있는
  16          향해
  17          보인다
  18          짓밟힌
  19          국민적
  20          저항에
  21          직면할
  22          정권이
  23          위해
  24          공수처법을
  25          제게
  26          대통령은
  27          '공수처는
  28          없는
  29          야당이
  30          공수처장
  31          사람들이
  32          민주당
  33          그러면서
  34          공수처가
  35          청와대와
  36          왜
  37          그
  38          인정하지
  39          이
  40          공수처를
  41          권력
  42          게
  43          정권의
  44          민주당이
  45          국회에서
  46          그걸
  47          권력은
  48          먼저
  49          주호영
  50        

In [202]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten,
                                     Concatenate,
                                     Lambda)
import matplotlib.pyplot as plt

noise를 token_table을 통해 text로 변환하는 변환기 구현

In [203]:

b = tf.constant([1,2,3,4,5,6,11,8,9,10])
c = tf.sort(b,axis=-1,direction='DESCENDING')
wh = tf.where(tf.math.greater_equal(b,c[5-1]))[:5]
wh

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[5],
       [6],
       [7],
       [8],
       [9]])>

In [204]:
import sys

def to_text(w):

    #tf.print(w)
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]

            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                try:
                    text += word_table[k]+' '
                except Exception as ex:
                    tf.print('to_text : word_table' + str(ex),k)
                
            texts.append(text)
    except Exception as ex:
        tf.print('to_text : ' + str(ex),sys.exc_info())

    return tf.constant(texts,dtype=tf.string)

In [205]:
# to_text 함수의 test
w = tf.random.normal([3,_NOISE_DIM])
print(w.shape)
e = to_text(w)
for t in e:
    print(t.numpy().decode('utf-8'))

(3, 394)
것 국민적 정권의 국회에서 공수처 등을 공수처법 개정을 돌입하겠다고 '패스트트랙'이라는 해보지 야당 것'이라고 비난을 꼬집었다 집권'의 동남권 군림하는 '반민주'를 곳간을 운용에서 것이다 요직에 사건이 불거져도 이해할 비리의 공수처장이 검사들과 임명하는 사법기구가 할지 경찰 이렇게 어떻게 환히 막을 밀어넣고 다시 정권은 
풀들이 국민적 저항에 공수처법을 청와대와 국민의힘 더불어민주당을 개정을 야당 임명에 개의치 대표가 탈원전과 시비를 청와대 민주당'이 다 운용에서 없이 사건이 공직자들을 것인데 하치장 임명하는 예측했다 수사를 추미애보다 오만함을 또 야당에게는 삭발하고 장외투쟁해 하나 속으로 국민은 풀이다 바람이 밭의 바람보다 일어서서 
왜 인정하지 권력 그걸 주호영 있다 야당 사람 좋아보이는 야당의 동의 상임이사국처럼 '비토권'을 우리를 탈원전과 대통령이 사업이니 거는 윤건영 언급했다 재정 괴물 가담할 분이 법치가 지도자를 목숨바쳐 정의를 정죄하는 저도 임명하는 두렵기만 검찰총장을 검찰의 비리 봉착하자 자리에는 내 온통 시인 


생성된 text의 embedding 변환기 구현<br>
embedding은 org_text의 embedding과 비교하여 원문과 유사하게 민들기 위한 목적

In [206]:
# 이거 이틀걸림...잘 몰라서 ㅈㄴ 헤맴

@tf.custom_gradient
def to_embedding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 10, 1e-3)  # gradient 증폭 
        dy_arr = tf.reshape(dy,(dy.shape[0],1024,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))
        return dy_arr_st

    #print(w)    
    texts = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)                 
        value = tf.constant(embedder.encode(texts,show_progress_bar=False),dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [207]:
# to_embedding 함수의 test
e = to_embedding(w)
for t in e:
    print(t.numpy())

[ 0.51557583 -0.23738354  0.65157902 ... -0.19659568 -0.01652551
 -0.03561371]
[ 0.83412939 -0.24076766  0.43435761 ... -0.55511433  0.12052266
  0.0986032 ]
[ 0.77625871 -0.11476165  0.5568496  ... -0.33783042  0.0517711
 -0.4096759 ]


In [208]:
# to_compression_ratio

@tf.custom_gradient
def to_compression_ratio(w):
    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 2, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))        
        return dy_arr_st

    const = tf.math.reduce_sum(tf.constant([x for x in range(tf.size(w[0]))],tf.int64)) 
    #print(const)
    value = None
    compression_ratio = [] #tf.Variable()
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            cr = tf.math.reduce_sum(tf.reshape(wh,(_MAX_GEN_TOKEN,)))
            compression_ratio.append(cr.numpy()/const.numpy())
        #print(compression_ratio)
        value = tf.constant(compression_ratio,dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad
    

In [209]:
# to_compression_ratio 함수의 test
e = to_compression_ratio(w)
for t in e:
    print(t.numpy())

0.09059557484403456
0.10734813551878689
0.09248136810426112


생성된 text의 morpheme code 변환기 구현<br>
morpheme code는 한국어 문장들(dataset)의 morpheme code와 비교하여 한국어 문법에 가깝게 만들기 위한 목적

In [210]:

@tf.custom_gradient
def to_morpcoding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 2, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],_MAX_MORP_LENGTH,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))
        return dy_arr_st

    #print(w)    
    codes = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)    
            
        for sentence in texts:
            code = morpheme_encode(sentence)
            if len(code) <= _MAX_MORP_LENGTH:
                codes.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])
            else:
                codes.append(code[:_MAX_MORP_LENGTH])
        value = tf.constant(codes,dtype=tf.int32)
    except Exception as ex:
        tf.print('to_morpcoding :' + str(ex),sys.exc_info())

    return value, grad

In [211]:
# to_morpcoding 함수의 test
e = to_morpcoding(w)
for t in e:
    print(t.numpy())

[ 2  2  9  2  5  2  5  7  7  2  2  5  7  7  2  2  5  2  6  3  2  2  3  5
  6  2  2  3  5  2  5  6  2  3  2  2  2  9  5  3  7  2  3  2  2  5  2  5
  2  5  7  2  5  2  5  6  2  6  2  5  7  7  2  9  2  9  5  2  6  2  7  2
  6  2 10  8  8  2  5  6  6  2  2  5  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 2  9  5  2  9  2  5  7  7  2  5  2  5  2  5  2  6  2  5  2  5  2  2  5
  2  2  2  5  2  2  5  2  5  2  2  3  2 10  2  5 10  2  5  2  9  5  2  5
  2  9  2  6  2  6  2  5  2  5  8  2  2  5  2  5  2  2  2  2  2  5  2  5
  2  5  2  5  2  5  2  5  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 2  2  6  2 10  2  8  2  2  8  6  2  5  2  2  5  3  2  9  3  5  2  5  2
  2  5  2  5  2  5  2  5  2  2  6  2  2  2  6  2  5  2  5  2  5  2  6  2
  5  2  9  5  2  5  2  6  8  2  5  2  5  2  2  6  2  5  2  2  2  0  0  0

Network 구성을 위해 사용자 정의 Layer 를 구현

In [212]:
# 이것도 잘 몰라서 하루 걸림... ㅜㅜ

from keras import backend as K
from keras.layers import Layer

#tf.executing_eagerly()

class Post_processing(Layer):

    def __init__(self, output_dim, encoder_func=None,Tout=tf.float64, **kwargs):
        self.output_dim = output_dim
        self.encoder = encoder_func
        self.Tout = Tout
        super(Post_processing, self).__init__(**kwargs)
    '''
    def build(self, input_shape):
        tf.print('build',input_shape)
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Post_processing, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오
    '''
    def call(self, input_data):
        #tf.print('Post_processing : call input_data',input_data.shape)
        value = tf.py_function(self.encoder,[input_data],Tout=self.Tout,name='encode_func')
        #print('value.shape:',value.shape)
        #value.set_shape((input_data.shape[0],self.output_dim))
        if self.output_dim > 0:
            value.set_shape((input_data.shape[0],self.output_dim))
        else:
            value.set_shape((input_data.shape[0],))
        #return tf.reshape(value,[input_data.shape[0]])  

        #value = tf.Variable((tf.zeros([input_data.shape[0],1024]) if self.Tout==tf.float64 else tf.zeros([input_data.shape[0],])),dtype=self.Tout,shape=( (input_data.shape[0],1024) if self.Tout==tf.float64 else (input_data.shape[0],)))
        #tf.py_function(self.encoder,[input_data],Tout=self.Tout)
        return value

    def compute_output_shape(self, input_shape):
        tf.print('compute_output_shape:',input_shape)
        input_shape = tensor_shape.TensorShape(input_shape).as_list()
        if self.output_dim > 0:
            return tensor_shape.TensorShape([input_shape[0], self.output_dim])
        return tensor_shape.TensorShape([input_shape[0]])

In [213]:
# 구성한 Layer의 test
e = Post_processing(1024,to_embedding,Tout=tf.float64)(w)
for c in e:
    print(c)

tf.Tensor(
[ 0.51557583 -0.23738354  0.65157902 ... -0.19659568 -0.01652551
 -0.03561371], shape=(1024,), dtype=float64)
tf.Tensor(
[ 0.83412939 -0.24076766  0.43435761 ... -0.55511433  0.12052266
  0.0986032 ], shape=(1024,), dtype=float64)
tf.Tensor(
[ 0.77625871 -0.11476165  0.5568496  ... -0.33783042  0.0517711
 -0.4096759 ], shape=(1024,), dtype=float64)


In [214]:
# 별로 중요하지는 않지만 Lambda layer를 활용하기 위한 assert 함수 구성
def assert_layer(input_data,out_dim=None):
    #tf.print(input_data)
    #print(input_data)
    assert input_data.shape[1] == out_dim
    return input_data

In [215]:
# 드디어 generator 구현
# 효과적으로 구성된 것인지는 모르겠음... 이것은 아직 많은 연구가 필요함.
# 또한 LSTM으로 바꾸어 길이의 한게를 극복해야 할 것...

def make_generator_model(org_words):
    input = Input(shape=(org_words,), dtype='float64') 
    x1 = Dense(org_words*2, use_bias=False)(input)
    #x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    x1 = Dense(org_words*4, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(max_length*total_words, use_bias=False, activation='tanh')(x1)
    x1 = Dense(org_words, use_bias=False)(x1)
    x1 = Lambda(assert_layer,arguments={'out_dim':org_words})(x1)
    #x1 = Reshape((max_length, total_words))(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = Softmax()(x1)        
    #x1 = MyCustomLayer(max_length*total_words)(x1)
    txt = Post_processing(0,to_text,Tout=tf.string)(x1)
    emb = Post_processing(1024,to_embedding,Tout=tf.float64)(x1)
    cmr = Post_processing(0,to_compression_ratio,Tout=tf.float64)(x1)
    cod = Post_processing(128,to_morpcoding,Tout=tf.int32)(x1)
    
    model = Model(input,[txt,emb,cmr,cod])
    
    model.summary()
    return model

generator = make_generator_model(_NOISE_DIM)

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, 394)]        0                                            
__________________________________________________________________________________________________
dense_49 (Dense)                (None, 788)          310472      input_27[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_35 (LeakyReLU)      (None, 788)          0           dense_49[0][0]                   
__________________________________________________________________________________________________
dense_50 (Dense)                (None, 1576)         1241888     leaky_re_lu_35[0][0]             
___________________________________________________________________________________________

In [216]:
# generator의 test
# Create a random noise and generate a sample
noise = tf.random.normal([3,_NOISE_DIM])
texts,embeddings,compratios,morpcodes = generator(noise, training=True)
print(texts.shape)
for i,txt in zip(range(len(texts)),texts):
    print(f" {i+1}> {txt.numpy().decode('utf-8')}" )
print(embeddings.shape)
print(compratios.shape)
print(morpcodes.shape)

(3,)
 1> 이제 공수처는 '공수처는 사람들이 인정하지 위한 정의당을 끌어들이기 선거법에 만들어낸 사람 행사할 '민주당 내건 부정하는 것이라고 법위에 앉아 이권을 반대하는지 이후 쓰레기 자신들의 지킴으로서 넘쳐 공수처장이 사법기구가 불을 앞장 수를 내 행태가 장외투쟁해 눈 이용한다는 아울러 다시 일어난다 힘을 광장에서 
 2> 보인다 인정하지 게 아우성 묶어 이어 원내대표인 사람 안보리 행사할 우리를 완성된다 탈원전과 시비를 출신 군림하는 재정 출범하면 권력이 챙기는 우려했다 분이 난다 신성을 수단으로 검사들과 임명하는 사법기구가 할지 검찰총장을 법무장관을 비리 저지하려다가 내 분명한 군사작전을 야당에게는 혼돈 바람이 일어서지 
 3> 주 대통령은 '공수처는 정권의 22일 건설 아우성 돌입하겠다고 꼼수 만들어낸 있는데 걱정하느냐고 '민주당 20년 안에 내건 있었다고 거는 민주주의를 군림하는 '반민주'를 곳간을 사라졌다 공직자들을 라며 공수처장이 할지 앞장 세워 출범시키려 막무가내 행태가 힘이 하나 온통 혼돈 아울러 바람이 고 정권은 
(3, 1024)
(3,)
(3, 128)


# Discriminator 구현

먼저 요약을 구분하기 위한 discriminator_summ 구현

In [217]:
# 문장에 대한 embeddings를 이용하여 org_text_emb (org_text의 embedding)과의 유사도를 계산한다.
import scipy

@tf.custom_gradient
def to_similarity(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 10, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],1024),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],1024))
        return dy_arr_st

    similarities = []
    value = None
    try:
        for embedding in w:
            distances = scipy.spatial.distance.cdist([embedding], [org_text_emb], "cosine")[0]
            similarities.append(distances[0])
            
        value = tf.constant(similarities,dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad


In [218]:
def make_discriminator_model():
    input_emb = Input(shape=(1024,), dtype='float64') 
    x1 = Post_processing(0,to_similarity,Tout=tf.float64)(input_emb)
    x1 = Reshape((1,))(x1)    
    #x1 = Dense(1024*2)(input_emb)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)

    
    input_cmp = Input(shape=(), dtype='float64') 
    imput_mrp = Input(shape=(_MAX_MORP_LENGTH,), dtype='int32')

    x2 = Reshape((1,))(input_cmp)
    x3 = Dense(256)(imput_mrp)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    x3 = Dense(256*3)(x3)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    concatted = Concatenate(axis=1)([x1, x2, x3])
    x4 = Flatten()(concatted)
    x4 = Dense(64, use_bias=False)(x4)
    #x4 = BatchNormalization()(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dense(1)(x4)
    
    model = Model([input_emb,input_cmp,imput_mrp],x4)
    
    model.summary()
    
    return model

discriminator = make_discriminator_model()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
dense_52 (Dense)                (None, 256)          33024       input_30[0][0]                   
__________________________________________________________________________________________________
input_28 (InputLayer)           [(None, 1024)]       0                                            
__________________________________________________________________________________________________
leaky_re_lu_37 (LeakyReLU)      (None, 256)          0           dense_52[0][0]                   
___________________________________________________________________________________________

In [219]:
# discriminator test

predict = discriminator([embeddings,compratios,morpcodes])
print(predict)

tf.Tensor(
[[-0.88418627]
 [-0.87208325]
 [-1.8090582 ]], shape=(3, 1), dtype=float32)


# GAN 을 이용한 loss 의 gradient 구현 --> 빡심

In [220]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [221]:
# 디짐
@tf.function
def train_step(real_embedding,real_morpcoding):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        # real에 가까우려면 discriminator의 학습은 real_embedding 이 zero (0)에 가깝게 학습시켜야 함.
        # 하지만 압축율의 개념으로는 본래는 ones (1)가 맞음.
        real_output = discriminator([real_embedding,np.zeros(len(real_embedding)),real_morpcoding], training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    #tf.print('train_step : gradients_of_discriminator=',gradients_of_discriminator)   
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        #real_output = discriminator(real_embedding, training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        #disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    #tf.print('train_step : gradients_of_generator=',gradients_of_generator)
 

    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    #tf.print('train_step : after discriminator_optimizer')    

In [222]:
EPOCHS = 5
# 요약문 생성의 확인을 위해 10개의 문장을 생성하고 train과정에서 각 epoch마다 변화를 확인한다.
seed = tf.random.normal([10, _NOISE_DIM])

In [223]:
# 생성된 문장의 원문 유사도를 측정하기 위한 함수

import scipy
#print(doc_emb)
def similarity_score(queries,org_embedding):

    total_score = 0
    query_embeddings = embedder.encode(queries,show_progress_bar=False)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], [org_embedding], "cosine")[0]
        results = zip(range(len(distances)), distances)
        for idx, distance in results:
            total_score += 1-distance
    return total_score

queries = []

texts,embeddings,compratios,morpcodes = generator(seed,training=False)

#count = 0
for t in texts:
    summary_text = t.numpy().decode('utf-8')
    queries.append(summary_text)
print('Similarity score:',str(similarity_score(queries,org_text_emb)))


Similarity score: 6.459584664565794


In [224]:
# Print iterations progress
class ProgressBar:
    # pb = ProgressBar(total=20, prefix = 'Epoch 1')
    def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill
        self.printEnd = printEnd
        self.ite = 0
    # pb.printProgress(1,'~~~~')
    def printProgress(self,iteration, text):
        self.ite += iteration
        percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

        filledLength = int(self.length * self.ite // self.total)
        bar = self.fill * filledLength + '-' * (self.length - filledLength)
        print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
        # Print New Line on Complete
        if self.ite == self.total: 
            print()

In [225]:
import time
from IPython import display # A command shell for interactive computing in Python.
import re

def train(dataset, epochs):
    print('Start with seed text.')
    seed = tf.random.normal([10, _NOISE_DIM])
    print('-------------------------------------------------------')
    texts,embeddings,compratios,morpcodes = generator(seed,training=False)
    for i,t in  zip(range(len(texts)),texts):
        summary_text = t.numpy().decode('utf-8')
        print(f'{i+1} > {summary_text}')        
    print('-------------------------------------------------------')
    print('')

    # A. For each epoch, do the following:
    for epoch in range(epochs):
        start = time.time()
        pb = ProgressBar(total=BATCH_COUNT, prefix = f'Epoch:{str(epoch+1)}/{epochs}')
        pb.printProgress(0,'Start batch.')
        # 1 - For each batch of the epoch, 
        for batch_num,(emb_batch_set,cod_batch_set) in zip(range(len(dataset)),dataset):
            # 1.a - run the custom "train_step" function
            # we just declared above
            #print(image_batch.shape)
            train_step(emb_batch_set,cod_batch_set)
            texts,embeddings,compratios,morpcodes = generator(seed,training=False)
            pb.printProgress(+1,f"Time for batch {batch_num + 1}/{BATCH_COUNT} is {int(time.time()-start)} sec, generated text:{texts[0].numpy().decode('utf-8')}")
        # 4 - Print out the completed epoch no. and the time spent
        #print (f'Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        #texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        count = 0
        queries = []
        texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        for i,t in  zip(range(len(texts)),texts):
            summary_text = t.numpy().decode('utf-8')
            print(f'{i+1} > {summary_text}')
            queries.append(summary_text)
            c = [m.start() for m in re.finditer(_MISMATCH_WORD, summary_text)]
            count += len(c)
        print(f'Mismatch count:{count} Similarity score:{str(similarity_score(queries,org_text_emb))}')
        print('')

In [228]:
if DO_ALL:
    EPOCHS = 30
    train(dataset, EPOCHS)

Start with seed text.
-------------------------------------------------------
1 > 며 보인다 게 정권의 치는 공수처법 묶어 야당 사람 안보리 탈원전과 내건 것은 출신 군림하는 '반민주'를 출범하면 거리낌 권부 요직에 가져가 없다'고 감시하는 특별감찰관은 종말 정부를 넘쳐 난다 세력을 마음대로 운영할지 내 환히 아닌 혼돈 아울러 바람이 눕는다 아우성치는 끝장내려는 
2 > 대통령과 야당이 인정하지 게 먼저 주호영 아우성 돌입하겠다고 놓고 만들어낸 있는데 완성된다 군림하는 '반민주'를 얻기 불법으로 이권을 했는데 분이 라며 권력형 수단으로 임명하는 할지 헌법과 운영할지 앞장 세워 저지하려다가 자리에는 내 앉힐 내일부터 행태가 라고 아닌 바람이 풀을 힘을 노래했다 
3 > 그 인정하지 게 정권의 아우성 치는 정의당을 꼼수 동의 없이는 유엔 상임이사국처럼 여당 동남권 있었다고 것이라고 출신 언급했다 괴물 사건을 버리면 것인데 이후 임명하지 신성을 수단으로 임명하는 위에 일 앞장 세워 내 겪어온 환히 개시하면 라고 장외투쟁해 규탄했다 다시 정권은 
4 > 대통령은 민주당이 건설 치는 위한 꼼수 불법·탈법으로 시행도 사람 없이는 얘기했고 상임이사국처럼 20년 대통령이 여기에 불법이 재정 괴물 범죄행위에 권부 사건을 공직자들을 처벌하는 분이 바로 넘쳐 수단으로 어떤 일을 경찰 앞장 세워 최전선에서 겪어온 라고 막을 속으로 이용한다는 정권은 지켜보게 
5 > 공수처는 보인다 저항에 직면할 왜 인정하지 건설 아우성 공수처법 위한 끌어들이기 묶어 안보리 상임이사국처럼 '민주당 탈원전과 윽박지른다 군림하는 얻기 요직에 각종 이권을 우려했다 것인데 임명하지 하치장 존재가 난다 수단으로 검사들과 사법기구로 운영할지 막무가내 분명한 행태가 라고 규탄했다 눕지만 정권은 아우성치는 
6 > 며 이제 공수처는 있는 저항에 민주당 민주당이 정권과 아우성 위한 고치려 사람 좋아보이는 야당의 행사할 완성된다 것은 민주주의를 출신 재

KeyboardInterrupt: ignored